## Streams Advanced Examples

In [1]:
import java.util.stream.Collectors;
import java.util.stream.Collector;
import java.util.stream.IntStream;

class Person {
    String name;
    int age;

    Person(String name, int age) {
        this.name = name;
        this.age = age;
    }
    
    String getName() {
        return this.name;
    }
    
    int getAge() {
        return this.age;
    }

    @Override
    public String toString() {
        return name;
    }
}

List<Person> persons =
    Arrays.asList(
        new Person("Max", 18),
        new Person("Peter", 23),
        new Person("Pamela", 23),
        new Person("David", 12));

class Bar {
    private String name;

    Bar(String name) {
        this.name = name;
    }
    
    String getName() {
        return this.name;
    }
}

class Foo {
    private String name;
    private List<Bar> bars = new ArrayList<>();

    Foo(String name) {
        this.name = name;
    }
    
    String getName() {
        return this.name;
    }
    
    List<Bar> getBars() {
        return this.bars;
    }
}

List<Foo> foos = new ArrayList<>();

IntStream
    .range(1, 4)
    .forEach(i -> foos.add(new Foo("Foo" + i)));

foos
    .forEach(f -> 
        IntStream
            .range(1, 4)
            .forEach(i -> 
                f.getBars().add(new Bar("Bar" + i + " <- " + f.getName()))));

### Streams.collect()

<code>Streams.collect()</code> is useful <i>terminal</i> operation to transform the elements of the stream into a different kind of result, e.g. a <code>List</code>, </code>Set</code> or <code>Map</code>:

In [2]:
persons
    .stream()
    .filter(p -> p.getName().startsWith("P"))
    .collect(Collectors.toList());

[Peter, Pamela]

For <code>Set</code> just use <code>Collectors.toSet()</code>:

In [3]:
persons
    .stream()
    .filter(p -> p.getName().startsWith("P"))
    .collect(Collectors.toSet());

[Peter, Pamela]

You can also use grouping by parameter:

In [4]:
persons
    .stream()
    .collect(Collectors.groupingBy(p -> p.getAge()))
    .forEach((age, p) -> System.out.format("age %s: %s\n", age, p));

age 18: [Max]
age 23: [Peter, Pamela]
age 12: [David]


You can also create aggregations on the elements of the stream, e.g. determining the average age of all persons:

In [5]:
persons
    .stream()
    .collect(Collectors.averagingInt(p -> p.getAge()));

19.0

Summarizing collectors return a special built-in summary statistics object:

In [6]:
persons
    .stream()
    .collect(Collectors.summarizingInt(p -> p.getAge()));

IntSummaryStatistics{count=4, sum=76, min=12, average=19.000000, max=23}

The next example joins all persons into a single string:

In [7]:
persons
    .stream()
    .filter(p -> p.getAge() >= 18)
    .map(p -> p.getName())
    .collect(Collectors.joining(" and ", "In Germany ", " are of legal age."));

In Germany Max and Peter and Pamela are of legal age.

The <code>join</code> collector accepts a delimiter as well as an optional prefix and suffix.

To transform the elements of the stream into a <code>Map</code> we have to specify how both the keys and the values should be mapped:

In [8]:
persons
    .stream()
    .collect(Collectors.toMap(
                Person::getAge,
                Person::getName,
                (name1, name2) -> name1 + ";" + name2));

{18=Max, 23=Peter;Pamela, 12=David}

You can also create your own collector.
For example we want to transform all persons of the stream into a single string consisting of all names in upper letters separated by the <code>|</code> pipe character: 

In [9]:
Collector<Person, StringJoiner, String> personNameCollector =
    Collector.of(
        () -> new StringJoiner(" | "),          // supplier
        (j, p) -> j.add(p.getName().toUpperCase()),  // accumulator
        (j1, j2) -> j1.merge(j2),               // combiner
        StringJoiner::toString);                // finisher

In [10]:
persons
    .stream()
    .collect(personNameCollector);

MAX | PETER | PAMELA | DAVID

Since strings in Java are immutable, we need a helper class like <code>StringJoiner</code> to let the collector construct our string. The <b>supplier</b> initially constructs such a <code>StringJoiner</code> with the appropriate delimiter. The <b>accumulator</b> is used to add each persons upper-cased name to the <code>StringJoiner</code>. The <b>combiner</b> knows how to merge two <code>StringJoiners</code> into one. In the last step the <b>finisher</b> constructs the desired <code>String</code> from the <code>StringJoiner</code>.

### Streams.flatMap()

<code>Streams.flatMap()</code> transforms each element of the stream into a stream of other objects. So each object will be transformed into zero, one or multiple other objects backed by streams. The contents of those streams will then be placed into the returned stream of the <code>flatMap</code> operation.

<code>Streams.flatMap()</code> accepts a function which has to return a stream of objects. So in order to resolve the <code>bar</code> objects of each <code>foo</code>, we just pass the appropriate function:

In [11]:
foos.stream()
    .flatMap(f -> f.getBars().stream())
    .forEach(b -> System.out.println(b.getName()));

Bar1 <- Foo1
Bar2 <- Foo1
Bar3 <- Foo1
Bar1 <- Foo2
Bar2 <- Foo2
Bar3 <- Foo2
Bar1 <- Foo3
Bar2 <- Foo3
Bar3 <- Foo3


Transformed the stream of three <code>foo</code> objects into a stream of nine <code>bar</code> objects.

Put it all together:

In [12]:
IntStream.range(1, 4)
    .mapToObj(i -> new Foo("Foo" + i))
    .peek(f -> IntStream.range(1, 4)
        .mapToObj(i -> new Bar("Bar" + i + " <- " + f.getName()))
        .forEach(f.getBars()::add))
    .flatMap(f -> f.getBars().stream())
    .forEach(b -> System.out.println(b.getName()));

Bar1 <- Foo1
Bar2 <- Foo1
Bar3 <- Foo1
Bar1 <- Foo2
Bar2 <- Foo2
Bar3 <- Foo2
Bar1 <- Foo3
Bar2 <- Foo3
Bar3 <- Foo3


### Streams.reduce()

The reduction operation combines all elements of the stream into a single result. Java8 supports three different kind of <code>Streams.reduce()</code> methods. 

The first <code>Streams.reduce()</code> reduces a stream of elements to exactly one element of the stream:

In [13]:
persons
    .stream()
    .reduce((p1, p2) -> p1.getAge() > p2.getAge() ? p1 : p2)
    .ifPresent(System.out::println);

Pamela


The <code>Streams.reduce()</code> method accepts a <code>BinaryOperator</code> accumulator function. That's actually a <code>BiFunction</code> where both operands share the same type, in that case <code>Person</code>. BiFunctions are like <code>Function</code> but accept two arguments. The example function compares both persons ages in order to return the person with the maximum age.

The second <code>Streams.reduce()</code> method accepts both an identity value and a <code>BinaryOperator</code> accumulator. 

This method can be utilized to construct a new Person with the aggregated names and ages from all other persons in the stream:

In [14]:
Person result = 
    persons
        .stream()
        .reduce(new Person("", 0), (p1, p2) -> {
                    p1.age += p2.age;
                    p1.name += p2.name;
                    return p1;
            });
            
result.getName();

MaxPeterPamelaDavid

In [15]:
result.getAge();

76

The third <code>Streams.reduce()</code> method accepts three parameters: an identity value, a <code>BiFunction</code> accumulator and a combiner function of type <code>BinaryOperator</code>:

In [16]:
persons
    .stream()
    .reduce(
        0, 
        (sum, p) -> sum += p.getAge(), 
        (sum1, sum2) -> sum1 + sum2
    );

76

In [17]:
persons
    .stream()
    .reduce(0,
        (sum, p) -> {
            System.out.format("accumulator: sum=%s; person=%s\n", sum, p);
            return sum += p.getAge();
        },
        (sum1, sum2) -> {
            System.out.format("combiner: sum1=%s; sum2=%s\n", sum1, sum2);
            return sum1 + sum2;
        });

accumulator: sum=0; person=Max
accumulator: sum=18; person=Peter
accumulator: sum=41; person=Pamela
accumulator: sum=64; person=David


76